## ✅ Roadmap del Proyecto de Web Scraping y Carga a Base de Datos

Este proyecto tiene como objetivo scrapear libros desde una web pública, obtener información adicional mediante una API externa y almacenar los datos en una base de datos relacional para su posterior análisis.

### 🛠️ Etapas del desarrollo

- [ ] **1. Web Scraping**
  - [x] Scrapear todos los géneros desde la página principal.
  - [x] Iterar sobre cada género para obtener todos los libros listados.
  - [ ] Extraer título, precio, stock, rating y link a la página del libro.
  - [ ] Extraer información detallada desde la página de cada libro (si es necesario).
  - [ ] Consultar la API de Google Books para obtener el autor y más detalles (usando el título).
  - [ ] Guardar los datos obtenidos en archivos `.csv` separados:
    - [ ] Libros (`libros.csv`)
    - [ ] Autores (`autores.csv`)
    - [ ] Géneros (`generos.csv`)

- [ ] **2. Validación**
  - [ ] Verificar la integridad y limpieza de los datos descargados.
  - [ ] Eliminar duplicados y manejar valores nulos si los hay.

- [ ] **3. Diseño de Base de Datos Relacional**
  - [ ] Crear script DDL:
    - [ ] Crear la base de datos (si no existe).
    - [ ] Crear las tablas `libros`, `autores`, `generos`, y tablas intermedias para relaciones N:N si aplica.
  - [ ] Crear diagrama UML/ER para visualizar relaciones.

- [ ] **4. Inserción de Datos**
  - [ ] Crear script DML para insertar los datos desde los CSV a la base de datos (usando Python + `psycopg2` o `SQLAlchemy` para PostgreSQL).
  - [ ] Comprobar inserciones correctas mediante queries de prueba.

- [ ] **5. Consultas y Análisis**
  - [ ] Escribir consultas SQL para:
    - [ ] Obtener todos los libros por género.
    - [ ] Buscar libros por autor.
    - [ ] Calcular estadísticas como precio promedio por género o autor.
  - [ ] (Opcional) Crear una vista para simplificar reportes.

---

> 📌 **Nota:** La base de datos será inicialmente verificada con SQLite por simplicidad, luego se adaptará a PostgreSQL para una implementación más robusta con PgAdmin.


In [ ]:
!pip install BeautifulSoup4
!pip install sqlalchemy psycopg2-binary pandas


lista generos
lista linklibro

In [3]:
# import web grabbing client and
# HTML parser
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd
from urllib.parse import urljoin

base_url = "http://books.toscrape.com/"

myurl = 'http://books.toscrape.com/index.html'
uClient = uReq(myurl)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, "html.parser")

bookshelf = page_soup.find("ul", {"class": "nav nav-list"})
links = bookshelf.find_all("a")

lista_generos = []
for i, link in enumerate(links):
    if i == 0:
        continue  # saltar 'Books'
    genero = link.get_text(strip=True)
    href = link.get("href")
    lista_generos.append({"nombre_genero": genero, "url": "http://books.toscrape.com/" + href})

# Guardar a CSV
generos_df = pd.DataFrame(lista_generos)
generos_df.insert(0, "id_genero", range(1, len(generos_df) + 1))
generos_df.to_csv("generos.csv", index=False)




lista_libros = []

for genero in lista_generos:
    genero_url = genero["url"]
    genero_nombre = genero["nombre_genero"]

    while genero_url:
        uClient = uReq(genero_url)
        html = uClient.read()
        uClient.close()

        genero_soup = soup(html, "html.parser")

        libros = genero_soup.find_all("article", class_="product_pod")
        for libro in libros:
            link_rel = libro.h3.a["href"]
            if "catalogue/" not in link_rel:
                link_rel = "catalogue/" + link_rel
            libro_url = "http://books.toscrape.com/" + link_rel
            lista_libros.append({
                "url": libro_url,
                "genero": genero_nombre
            })

        # siguiente página (si existe)
        next_btn = genero_soup.find("li", class_="next")
        if next_btn:
            next_url = next_btn.a["href"]
            genero_url = "/".join(genero_url.split("/")[:-1]) + "/" + next_url
        else:
            genero_url = None

'''
libros = ("Titulos.csv")
f = open(libros, "w")
headers = "Titulos\n"
f.write(headers)
'''
#guardar en un diccionario los campos de los libros
libros_data = []

for i, libro in enumerate(lista_libros):
    try:
        link_rel = libro.h1["href"]
        libro_url = urljoin(base_url + "catalogue/", link_rel)
        uClient = uReq(libro_url["url"])
        html = uClient.read()
        uClient.close()
        soup_libro = soup(html, "html.parser")

        titulo = soup_libro.find("h1").text.strip()
        precio = soup_libro.find("p", class_="price_color").text.strip().lstrip("£")
        stock = soup_libro.find("p", class_="instock availability").text.strip()
        
        rating_str = soup_libro.find("p", class_="star-rating")["class"][1]
        rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
        rating = rating_map.get(rating_str, 0)

        # Autor desde breadcrumb
        breadcrumb = soup_libro.find("ul", class_="breadcrumb").find_all("li")
        autor = breadcrumb[2].text.strip()

        libros_data.append({
            "titulo": titulo,
            "precio": precio,
            "stock": stock,
            "rating": rating,
            "autor": autor,
            "genero": libro["genero"],
            "url": libro["url"]
        })

    except Exception as e:
        print(f"Error en libro {i}: {e}")

# asociar ids
genero_id_map = {nombre: id_ for id_, nombre in zip(generos_df['nombre_genero'], generos_df['id_genero'])}

libros_final = []
for i, l in enumerate(libros_data, start=1):
    libros_final.append({
        "id": i,
        "titulo": l["titulo"],
        "precio": l["precio"],
        "stock": l["stock"],
        "rating": l["rating"],
        "id_genero": genero_id_map[l["genero"]],
        "url": l["url"]
    })

libros_df = pd.DataFrame(libros_final)
libros_df.to_csv("libros.csv", index=False)




Error en libro 0: 'dict' object has no attribute 'h1'
Error en libro 1: 'dict' object has no attribute 'h1'
Error en libro 2: 'dict' object has no attribute 'h1'
Error en libro 3: 'dict' object has no attribute 'h1'
Error en libro 4: 'dict' object has no attribute 'h1'
Error en libro 5: 'dict' object has no attribute 'h1'
Error en libro 6: 'dict' object has no attribute 'h1'
Error en libro 7: 'dict' object has no attribute 'h1'
Error en libro 8: 'dict' object has no attribute 'h1'
Error en libro 9: 'dict' object has no attribute 'h1'
Error en libro 10: 'dict' object has no attribute 'h1'
Error en libro 11: 'dict' object has no attribute 'h1'
Error en libro 12: 'dict' object has no attribute 'h1'
Error en libro 13: 'dict' object has no attribute 'h1'
Error en libro 14: 'dict' object has no attribute 'h1'
Error en libro 15: 'dict' object has no attribute 'h1'
Error en libro 16: 'dict' object has no attribute 'h1'
Error en libro 17: 'dict' object has no attribute 'h1'
Error en libro 18: '

agregar "https://books.toscrape.com/" a las direcciones

traer datos de libros

for i in libro:
    Titulo
    direccion = i.select("h3 > a")#iterar los libros
        lista_libro.append(direccion)
        for j in direccion:
            f.write(j.get_text() + "\n")
    Precio
    precio = i.select("p[class="price-color"]")

    Stock
    precio = i.select("p[class="instock"]")
    Rating

    Descripcion

    UPC
